# Ada Academy Application

## September 11, 2017
## Melissa Alegre

### Overview

This notebook contains the code in used answering the questions in 'Step 3: Assessment' for the application to the Ada Academy.

### Data Nomenclature Cleaning

After inspecting the csv file, I observed that the file contains several rows of text before the actual data. It is clear that certain column names are used multiple times (e.g. # Disbursements). My approach will be to drop the irrelevant rows (first 5) and rename the columns to reflect the given category (e.g. 'DL Subsidized - Undergraduate' etc.). 

I also observed that the 2015 file differs slightly from the 2010 file: in 2010, the 'DL SUBSIDIZED' category was split into graduate and undergraduate. For 2015, the category was no longer split. The column names will reflect this difference.

In [1]:
#import libraries

import pandas as pd
import numpy as np
import re

# change settings to be able to view more columns of the data (the csvs have 30+ columns)

pd.options.display.max_columns = 50

In [2]:
#load csv data into pandas data frames

data_2010 = pd.read_csv('DL_Dashboard_AY2010_2011_Q1.csv', skiprows=5)
data_2015 = pd.read_csv('DL_Dashboard_AY2015_2016_Q1.csv',skiprows=5)

#determine length of files

print "2010-2011 data file length:", len(data_2010)
print "2015-2016 data file length:", len(data_2015)

#view the top rows of one file

print data_2010.head(3)

2010-2011 data file length: 5597
2015-2016 data file length: 4797
     OPE ID                          School State Zip Code School Type  \
0  106100.0       ALASKA PACIFIC UNIVERSITY    AK    99508     PRIVATE   
1  106300.0  UNIVERSITY OF ALASKA FAIRBANKS    AK    99775      PUBLIC   
2  106500.0  UNIVERSITY OF ALASKA SOUTHEAST    AK    99801      PUBLIC   

  Recipients # of Loans Originated $ of Loans Originated # of Disbursements  \
0       234                   235         $1,029,839.00                 235   
1     1,395                 1,435         $5,199,649.00               1,435   
2       375                   385         $1,419,529.00                 385   

  $ of Disbursements Recipients.1 # of Loans Originated.1  \
0       $543,444.00          106                     106    
1     $2,865,794.00          218                     228    
2       $764,945.00           91                      94    

  $ of Loans Originated.1 # of Disbursements.1 $ of Disbursements.1  \
0   

In [3]:
'''
The following column nomenclature will be used to refect the various categories:

dls = DL SUBSIDIZED (2016 only - undergraduate / graduate are not separated)
dlsu = DL SUBSIDIZED- UNDERGRADUATE (2011 only)
dlsg = DL SUBSIDIZED- GRADUATE (2011 only)
dluu = DL UNSUBSIDIZED- UNDERGRADUATE 
dlug = DL UNSUBSIDIZED- GRADUATE
dlpp = DL PARENT PLUS  
dlgp = DL GRAD PLUS

The following abbreviations will be used for column headers:

rec = Recipients
nlo = # of Loans Originated
dlo = $ of Loans Originated
nd = # of Disbursements
dd = $ of Disbursements
'''

#rename columns for both dataframes using the nomenclatures outlined above

data_2010.rename(columns={data_2010.columns[5]:'dlsu_rec',data_2010.columns[6]:'dlsu_nlo',
                 data_2010.columns[7]:'dlsu_dlo',data_2010.columns[8]:'dlsu_nd',data_2010.columns[9]:'dlsu_dd',
                         data_2010.columns[10]:'dlsg_rec',data_2010.columns[11]:'dlsg_nlo',
                 data_2010.columns[12]:'dlsg_dlo',data_2010.columns[13]:'dlsg_nd',data_2010.columns[14]:'dlsg_dd',
                          data_2010.columns[15]:'dluu_rec',data_2010.columns[16]:'dluu_nlo',
                 data_2010.columns[17]:'dluu_dlo',data_2010.columns[18]:'dluu_nd',data_2010.columns[19]:'dluu_dd',
                          data_2010.columns[20]:'dlug_rec',data_2010.columns[21]:'dlug_nlo',
                 data_2010.columns[22]:'dlug_dlo',data_2010.columns[23]:'dlug_nd',data_2010.columns[24]:'dlug_dd',
                          data_2010.columns[25]:'dlpp_rec',data_2010.columns[26]:'dlpp_nlo',
                 data_2010.columns[27]:'dlpp_dlo',data_2010.columns[28]:'dlpp_nd',data_2010.columns[29]:'dlpp_dd',
                          data_2010.columns[30]:'dlgp_rec',data_2010.columns[31]:'dlgp_nlo',
                 data_2010.columns[32]:'dlgp_dlo',data_2010.columns[33]:'dlgp_nd',data_2010.columns[34]:'dlgp_dd'},
                 inplace=True)

data_2015.rename(columns={data_2015.columns[5]:'dls_rec',data_2015.columns[6]:'dls_nlo',
                 data_2015.columns[7]:'dls_dlo',data_2015.columns[8]:'dls_nd',data_2015.columns[9]:'dls_dd',
                          data_2015.columns[10]:'dluu_rec',data_2015.columns[11]:'dluu_nlo',
                 data_2015.columns[12]:'dluu_dlo',data_2015.columns[13]:'dluu_nd',data_2015.columns[14]:'dluu_dd',
                          data_2015.columns[15]:'dlug_rec',data_2015.columns[16]:'dlug_nlo',
                 data_2015.columns[17]:'dlug_dlo',data_2015.columns[18]:'dlug_nd',data_2015.columns[19]:'dlug_dd',
                          data_2015.columns[20]:'dlpp_rec',data_2015.columns[21]:'dlpp_nlo',
                 data_2015.columns[22]:'dlpp_dlo',data_2015.columns[23]:'dlpp_nd',data_2015.columns[24]:'dlpp_dd',
                          data_2015.columns[25]:'dlgp_rec',data_2015.columns[26]:'dlgp_nlo',
                 data_2015.columns[27]:'dlgp_dlo',data_2015.columns[28]:'dlgp_nd',data_2015.columns[29]:'dlgp_dd'},
                 inplace=True)

In [4]:
# confirm that the new column headers look reasonable

print data_2010.head(2)
print '\n'
print data_2015.head(2)

     OPE ID                          School State Zip Code School Type  \
0  106100.0       ALASKA PACIFIC UNIVERSITY    AK    99508     PRIVATE   
1  106300.0  UNIVERSITY OF ALASKA FAIRBANKS    AK    99775      PUBLIC   

  dlsu_rec dlsu_nlo         dlsu_dlo dlsu_nd          dlsu_dd dlsg_rec  \
0     234      235    $1,029,839.00      235     $543,444.00      106    
1   1,395    1,435    $5,199,649.00    1,435   $2,865,794.00      218    

  dlsg_nlo         dlsg_dlo dlsg_nd        dlsg_dd dluu_rec dluu_nlo  \
0     106      $807,462.00      106   $440,993.00      229      231    
1     228    $1,533,289.00      228   $820,725.00    1,418    1,489    

          dluu_dlo dluu_nd          dluu_dd dlug_rec dlug_nlo  \
0   $1,130,540.00      231     $592,659.00       91       91    
1   $6,693,304.00    1,489   $3,672,335.00      155      162    

          dlug_dlo dlug_nd        dlug_dd dlpp_rec dlpp_nlo       dlpp_dlo  \
0     $952,079.00       91   $512,270.00       37       37    $

The columns look reasonable. The application states that I should exclude all rows where:

* The school is designated as a foreign institution.
* The final digit of the school's ZIP code is 3, 5, or 7.

In [5]:
# exclude rows where school is a foreign institution

# get the names of the various school type options

print "2010 school types:\n", data_2010['School Type'].value_counts()
print '\n'
print "2015 school types:\n", data_2015['School Type'].value_counts()

2010 school types:
PROPRIETARY           1789
PRIVATE               1524
PUBLIC                1383
FOREIGN PUBLIC         169
FOREIGN PRIVATE         49
FOREIGN FOR-PROFIT       4
Name: School Type, dtype: int64


2015 school types:
Proprietary           1735
Private-Nonprofit     1510
Public                1318
Foreign-Public         166
Foreign-Private         61
Foreign-For-Profit       7
Name: School Type, dtype: int64


In [6]:
# drop rows containing foreign institutions

print "2010-2011 data file length BEFORE dropping foreign schools:", len(data_2010)
print "2015-2016 data file length BEFORE dropping foreign schools:",len(data_2015)

data_2010 = data_2010[(data_2010['School Type'] == 'PROPRIETARY') | (data_2010['School Type'] == 'PRIVATE') |
                      (data_2010['School Type'] == 'PUBLIC')]

data_2015 = data_2015[(data_2015['School Type'] == 'Proprietary') | (data_2015['School Type'] == 'Private-Nonprofit') |
                      (data_2015['School Type'] == 'Public')]

print "2010-2011 data file length AFTER dropping foreign schools:", len(data_2010)
print "2015-2016 data file length AFTER dropping foreign schools:", len(data_2015)

2010-2011 data file length BEFORE dropping foreign schools: 5597
2015-2016 data file length BEFORE dropping foreign schools: 4797
2010-2011 data file length AFTER dropping foreign schools: 4696
2015-2016 data file length AFTER dropping foreign schools: 4563


In [7]:
# exclude rows where zip code ends in 3,5 or 7

# create a small function to identify whether the last digit is a 3,5,7

def last_digit(x):
    if x[-1] == '3':
        y = 1
    elif x[-1] == '5':
        y = 1
    elif x[-1] == '7':
        y = 1
    else:
        y = 0
    return y

# create a new column in each dataframe to identify zip codes that match the criteria

data_2010['zip_check'] = data_2010['Zip Code'].apply(last_digit)
data_2015['zip_check'] = data_2015['Zip Code'].apply(last_digit)

# filter out any rows where the value of zip_check is 1 (i.e. this means the zip ends in 3,5 or 7)

print "2010-2011 data file length BEFORE dropping schools with zips ending in 3,5,7:",len(data_2010)
print "2015-2016 data file length BEFORE dropping schools with zips ending in 3,5,7:",len(data_2015)

data_2010 = data_2010[data_2010['zip_check'] == 0]
data_2015 = data_2015[data_2015['zip_check'] == 0]

print "2010-2011 data file length AFTER dropping schools with zips ending in 3,5,7:",len(data_2010)
print "2015-2016 data file length AFTER dropping schools with zips ending in 3,5,7:",len(data_2015)

2010-2011 data file length BEFORE dropping schools with zips ending in 3,5,7: 4696
2015-2016 data file length BEFORE dropping schools with zips ending in 3,5,7: 4563
2010-2011 data file length AFTER dropping schools with zips ending in 3,5,7: 3401
2015-2016 data file length AFTER dropping schools with zips ending in 3,5,7: 3315


I can now start addressing the 5 application questions

### QUESTION 1

Consider all of the schools that disbursed a total of greater than or equal to 2,000 USD and less than 9,500 USD in loans for the time period reported on the spreadsheets provided (i.e. quarter 1). How many more schools met this criteria in 2015 than in 2010?

In [8]:
# create a new column that totals up the disbursed loan amounts

# first remove $ signs and commas from the dataframe and convert '-   ' to 0

data_2010.replace('\$','',regex=True,inplace=True)
data_2015.replace('\$','',regex=True,inplace=True)
data_2010.replace(',','',regex=True,inplace=True)
data_2015.replace(',','',regex=True,inplace=True)
data_2010.replace('-   ',0,regex=True,inplace=True)
data_2015.replace('-   ',0,regex=True,inplace=True)

In [9]:
# print a list of column names for easy copy / paste into a list of columns to convert to numbers

list(data_2010)

['OPE ID',
 'School',
 'State',
 'Zip Code',
 'School Type',
 'dlsu_rec',
 'dlsu_nlo',
 'dlsu_dlo',
 'dlsu_nd',
 'dlsu_dd',
 'dlsg_rec',
 'dlsg_nlo',
 'dlsg_dlo',
 'dlsg_nd',
 'dlsg_dd',
 'dluu_rec',
 'dluu_nlo',
 'dluu_dlo',
 'dluu_nd',
 'dluu_dd',
 'dlug_rec',
 'dlug_nlo',
 'dlug_dlo',
 'dlug_nd',
 'dlug_dd',
 'dlpp_rec',
 'dlpp_nlo',
 'dlpp_dlo',
 'dlpp_nd',
 'dlpp_dd',
 'dlgp_rec',
 'dlgp_nlo',
 'dlgp_dlo',
 'dlgp_nd',
 'dlgp_dd',
 'zip_check']

In [10]:
#repeat for 2015

list(data_2015)

['OPE ID',
 'School',
 'State',
 'Zip Code',
 'School Type',
 'dls_rec',
 'dls_nlo',
 'dls_dlo',
 'dls_nd',
 'dls_dd',
 'dluu_rec',
 'dluu_nlo',
 'dluu_dlo',
 'dluu_nd',
 'dluu_dd',
 'dlug_rec',
 'dlug_nlo',
 'dlug_dlo',
 'dlug_nd',
 'dlug_dd',
 'dlpp_rec',
 'dlpp_nlo',
 'dlpp_dlo',
 'dlpp_nd',
 'dlpp_dd',
 'dlgp_rec',
 'dlgp_nlo',
 'dlgp_dlo',
 'dlgp_nd',
 'dlgp_dd',
 'Unnamed: 30',
 'zip_check']

In [11]:
# convert columns of interest to numbers

cols_2010 = ['Zip Code','dlsu_rec',
 'dlsu_nlo',
 'dlsu_dlo',
 'dlsu_nd',
 'dlsu_dd',
 'dlsg_rec',
 'dlsg_nlo',
 'dlsg_dlo',
 'dlsg_nd',
 'dlsg_dd',
 'dluu_rec',
 'dluu_nlo',
 'dluu_dlo',
 'dluu_nd',
 'dluu_dd',
 'dlug_rec',
 'dlug_nlo',
 'dlug_dlo',
 'dlug_nd',
 'dlug_dd',
 'dlpp_rec',
 'dlpp_nlo',
 'dlpp_dlo',
 'dlpp_nd',
 'dlpp_dd',
 'dlgp_rec',
 'dlgp_nlo',
 'dlgp_dlo',
 'dlgp_nd',
 'dlgp_dd']

cols_2015 = ['Zip Code','dls_rec',
 'dls_nlo',
 'dls_dlo',
 'dls_nd',
 'dls_dd',
 'dluu_rec',
 'dluu_nlo',
 'dluu_dlo',
 'dluu_nd',
 'dluu_dd',
 'dlug_rec',
 'dlug_nlo',
 'dlug_dlo',
 'dlug_nd',
 'dlug_dd',
 'dlpp_rec',
 'dlpp_nlo',
 'dlpp_dlo',
 'dlpp_nd',
 'dlpp_dd',
 'dlgp_rec',
 'dlgp_nlo',
 'dlgp_dlo',
 'dlgp_nd',
 'dlgp_dd']

data_2010.ix[:, cols_2010] = data_2010.ix[:, cols_2010].apply(pd.to_numeric)
data_2015.ix[:, cols_2015] = data_2015.ix[:, cols_2015].apply(pd.to_numeric)

In [12]:
# confirm the data types

print "2010 data column types:\n",data_2010.dtypes
print "2015 data column types:\n",data_2015.dtypes

2010 data column types:
OPE ID         float64
School          object
State           object
Zip Code         int64
School Type     object
dlsu_rec         int64
dlsu_nlo         int64
dlsu_dlo       float64
dlsu_nd          int64
dlsu_dd        float64
dlsg_rec         int64
dlsg_nlo         int64
dlsg_dlo       float64
dlsg_nd          int64
dlsg_dd        float64
dluu_rec         int64
dluu_nlo         int64
dluu_dlo       float64
dluu_nd          int64
dluu_dd        float64
dlug_rec         int64
dlug_nlo         int64
dlug_dlo       float64
dlug_nd          int64
dlug_dd        float64
dlpp_rec         int64
dlpp_nlo         int64
dlpp_dlo       float64
dlpp_nd          int64
dlpp_dd        float64
dlgp_rec         int64
dlgp_nlo         int64
dlgp_dlo       float64
dlgp_nd          int64
dlgp_dd        float64
zip_check        int64
dtype: object
2015 data column types:
OPE ID           int64
School          object
State           object
Zip Code         int64
School Type     ob

In [13]:
# Create a column that sums up all the dispursed amounts for each school for each year

data_2010['total_disp'] = (data_2010['dlsu_dd'] + data_2010['dlsg_dd'] + data_2010['dluu_dd']
                           + data_2010['dlug_dd'] + data_2010['dlpp_dd'] + data_2010['dlgp_dd'])

data_2015['total_disp'] = (data_2015['dls_dd'] + data_2015['dluu_dd']
                           + data_2015['dlug_dd'] + data_2015['dlpp_dd'] + data_2015['dlgp_dd'])

In [18]:
# Answer the question: find a count for each of the two years of the number of schools that disbursed a total amount
# >= $2,000 and less than $9,500

print "Schools with $2,000 <= total dispursed < $,9000 (2010):"
print (len (data_2010[(data_2010['total_disp'] >= 2000) & (data_2010['total_disp'] < 9500)]))

print "Schools with $2,000 <= total dispursed < $,9000 (2015):"
print (len (data_2015[(data_2015['total_disp'] >= 2000) & (data_2015['total_disp'] < 9500)]))

Schools with $2,000 <= total dispursed < $,9000 (2010):
145
Schools with $2,000 <= total dispursed < $,9000 (2015):
151


### QUESTION 1 ANSWER

6 (151 minus 145) more schools met the criteria in 2015 vs. 2010.

### QUESTION 2

Consider the sum of expected total loan amount if the loans were fully disbursed for each school. In 2010 as compared to 2015, for how many more schools was this amount greater than $20,000,000?

In [15]:
# Create a column that sums up all the expected loan amounts for each school for each year

data_2010['total_loan_amt'] = (data_2010['dlsu_dlo'] + data_2010['dlsg_dlo'] + data_2010['dluu_dlo']
                           + data_2010['dlug_dlo'] + data_2010['dlpp_dlo'] + data_2010['dlgp_dlo'])

data_2015['total_loan_amt'] = (data_2015['dls_dlo'] + data_2015['dluu_dlo']
                           + data_2015['dlug_dlo'] + data_2015['dlpp_dlo'] + data_2015['dlgp_dlo'])

In [16]:
# Do a 'sanity check' on the total numbers for each year:

print (data_2010['total_disp'].describe())
print (data_2015['total_disp'].describe())
print (data_2010['total_loan_amt'].describe())
print (data_2015['total_loan_amt'].describe())

count    3.401000e+03
mean     6.641916e+06
std      1.805718e+07
min      6.200000e+01
25%      1.161010e+05
50%      9.295180e+05
75%      5.382443e+06
max      3.575169e+08
Name: total_disp, dtype: float64
count    3.315000e+03
mean     6.701368e+06
std      1.693591e+07
min      2.220000e+02
25%      8.073700e+04
50%      7.666100e+05
75%      5.234610e+06
max      2.466167e+08
Name: total_disp, dtype: float64
count    3.401000e+03
mean     1.273155e+07
std      3.378042e+07
min      1.240000e+02
25%      2.229590e+05
50%      1.906502e+06
75%      1.012131e+07
max      6.235302e+08
Name: total_loan_amt, dtype: float64
count    3.315000e+03
mean     1.260603e+07
std      3.100347e+07
min      4.430000e+02
25%      1.577915e+05
50%      1.464339e+06
75%      1.009592e+07
max      4.540196e+08
Name: total_loan_amt, dtype: float64


In [19]:
# Answer the question: find a count for each of the two years of the number of schools that had an expected total
# loan amount in excess of $20,000,000

print "Schools with > $20MM expected total loan amount (2010):"
print (len (data_2010[(data_2010['total_loan_amt'] > 20000000)]))

print "Schools with > $20MM expected total loan amount (2015):"
print (len (data_2015[(data_2015['total_loan_amt'] > 20000000)]))

Schools with > $20MM expected total loan amount (2010):
506
Schools with > $20MM expected total loan amount (2015):
505


### QUESTION 2 ANSWER

1 (506 minus 505) more school met the criteria in 2010 vs. 2015.

### QUESTION 3

In 2015, consider all the colonial colleges founded before the declaration of independence. Amongst these, what was the largest number of recipients within a school for either DL Graduate or DL Grad Plus loans?

In [20]:
'''
A list of the 9 colonial colleges can be found at:
https://en.wikipedia.org/wiki/Colonial_colleges
'''

# create a list of the colonial colleges

colonial = ['HARVARD UNIVERSITY','COLLEGE OF WILLIAM & MARY','YALE UNIVERSITY','PRINCETON UNIVERSITY',
            'COLUMBIA UNIVERSITY','UNIVERSITY OF PENNSYLVANIA','BROWN UNIVERSITY',
            'RUTGERS, THE STATE UNIVERSITY OF NEW JERSEY','DARTMOUTH COLLEGE']

# loop over the list and find the number of graduate recipients for each school

for school in colonial:
    print school
    print data_2015['dlug_rec'][data_2015['School'] == school]
    print data_2015['dlgp_rec'][data_2015['School'] == school]

HARVARD UNIVERSITY
721    2254
Name: dlug_rec, dtype: int64
721    1221
Name: dlgp_rec, dtype: int64
COLLEGE OF WILLIAM & MARY
Series([], Name: dlug_rec, dtype: int64)
Series([], Name: dlgp_rec, dtype: int64)
YALE UNIVERSITY
254    1677
Name: dlug_rec, dtype: int64
254    726
Name: dlgp_rec, dtype: int64
PRINCETON UNIVERSITY
1000    16
Name: dlug_rec, dtype: int64
1000    5
Name: dlgp_rec, dtype: int64
COLUMBIA UNIVERSITY
Series([], Name: dlug_rec, dtype: int64)
Series([], Name: dlgp_rec, dtype: int64)
UNIVERSITY OF PENNSYLVANIA
1439    2886
Name: dlug_rec, dtype: int64
1439    1506
Name: dlgp_rec, dtype: int64
BROWN UNIVERSITY
1453    299
Name: dlug_rec, dtype: int64
1453    83
Name: dlgp_rec, dtype: int64
RUTGERS, THE STATE UNIVERSITY OF NEW JERSEY
Series([], Name: dlug_rec, dtype: int64)
Series([], Name: dlgp_rec, dtype: int64)
DARTMOUTH COLLEGE
Series([], Name: dlug_rec, dtype: int64)
Series([], Name: dlgp_rec, dtype: int64)


### QUESTION 3 ANSWER

By inspecting the above list, the largest number of recipients of DL Graduate or Graduate Plus loans in 2015 occured for the University of Pennsylvania, which had 2,886 DL Unsubsidized - Graduate loans that year.

NOTE: 4 of the 9 colleges were not even considered, due to the zip code requirement established for this analysis (i.e. no zip codes ending in 3,5, or 7)

### QUESTION 4

In 2015, consider all the private nonprofit schools in King county, WA. For these schools, consider the expected total loan amount if the loan is fully disbursed for unsubsidized graduate studies. Exclude all schools where the unsubsidized graduate loan amount is not available i.e. “-” or 0. What was the median value?

In [21]:
'''
Zip codes are the best way to find all King County, WA schools. The following site provides a list of King County
zip codes:

http://www.zipcodestogo.com/King/WA/

king_zip = [98001,98002,98003,98004,98005,98006,98007,98008,98009,98010,98011,98013,98014,98015,98019,98022,98023,
98024,98025,98027,98028,98029,98030,98031,98032,98033,98034,98035,98038,98039,98040,98041,98042,98045,98047,
98050,98051,98052,98053,98054,98055,98056,98057,98058,98059,98062,98063,98064,98065,98070,98071,98072,98073,
98074,98075,98077,98083,98089,98092,98093,98101,98102,98103,98104,98105,98106,98107,98108,98109,98111,98112,
98113,98114,98115,98116,98117,98118,98119,98121,98122,98124,98125,98126,98127,98129,98131,98132,98133,98134,98136,
98138,98139,98141,98144,98145,98146,98148,98151,98154,98155,98158,98160,98161,98164,98165,98166,98168,98170,98171,
98174,98175,98177,98178,98181,98184,98185,98188,98190,98191,98194,98195,98198,98199,98224,98288]

'''

# write a function to identify King County schools

def king(row):
    
    king_zip = [98001,98002,98003,98004,98005,98006,98007,98008,98009,98010,98011,98013,98014,98015,98019,98022,
                98023,98024,98025,98027,98028,98029,98030,98031,98032,98033,98034,98035,98038,98039,98040,98041,
                98042,98045,98047,98050,98051,98052,98053,98054,98055,98056,98057,98058,98059,98062,98063,98064,
                98065,98070,98071,98072,98073,98074,98075,98077,98083,98089,98092,98093,98101,98102,98103,98104,
                98105,98106,98107,98108,98109,98111,98112,98113,98114,98115,98116,98117,98118,98119,98121,98122,
                98124,98125,98126,98127,98129,98131,98132,98133,98134,98136,98138,98139,98141,98144,98145,98146,
                98148,98151,98154,98155,98158,98160,98161,98164,98165,98166,98168,98170,98171,98174,98175,98177,
                98178,98181,98184,98185,98188,98190,98191,98194,98195,98198,98199,98224,98288]
    
    if row['Zip Code'] in king_zip:
        y = 1
    else:
        y = 0
    return y

data_2015['king'] = data_2015.apply(king,axis=1)

# filter the 2015 dataframe for private nonprofit, King County schools

data_2015_king = data_2015[(data_2015['king'] == 1) & (data_2015['School Type'] == 'Private-Nonprofit')]
print data_2015_king

       OPE ID                                           School State  \
1735   378800                       SEATTLE PACIFIC UNIVERSITY    WA   
1736   379000                               SEATTLE UNIVERSITY    WA   
2727  1231500                      CORNISH COLLEGE OF THE ARTS    WA   
2791  1302200                       CITY UNIVERSITY OF SEATTLE    WA   
3118  2242500                                BASTYR UNIVERSITY    WA   
3914  3466400  SEATTLE SCHOOL OF THEOLOGY AND PSYCHOLOGY (THE)    WA   

      Zip Code        School Type  dls_rec  dls_nlo    dls_dlo  dls_nd  \
1735     98119  Private-Nonprofit     1511     1574  6648800.0    1574   
1736     98122  Private-Nonprofit     1537     1537  6796469.0    1538   
2727     98121  Private-Nonprofit      454      464  2017502.0     464   
2791     98121  Private-Nonprofit      188      196   655667.0     196   
3118     98028  Private-Nonprofit      164      167   827441.0     209   
3914     98121  Private-Nonprofit        0        0

In [22]:
# the value of interest is the 'dlug_dlo' field (i.e. DL unsubsidized graduate - $ loan originated)
# exclude any schools with a value of zero and find the median of those remaining - use pandas 'median' function

data_2015_king[data_2015_king['dlug_dlo'] > 0]['dlug_dlo'].median()

5889721.0

### QUESTION 4 ANSWER

The median value for the field of interest is $5,889,721

### QUESTION 5

Consider all the schools who were football champions in National Collegiate Athletic Association (reference: www.ncaa.com) from 2001 to 2015 (both years included) in the FBS division. For these schools (only include main campuses that participate in football and not all campuses) consider the expected total loan amount if the loan is fully disbursed for GRAD PLUS loans in 2015 data provided. Calculate the sum of these amounts.

In [23]:
'''
The list of FBS champions is at:

http://www.ncaa.com/history/football/fbs

Several schools repeat (i.e. they have won multiple times). 
Generate a list of the champions and cross check the list against the data files to ensure
the names used are in the correct format for the main campus:

champs = ['UNIVERSITY OF ALABAMA','AUBURN UNIVERSITY','UNIVERSITY OF FLORIDA','FLORIDA STATE UNIVERSITY',
'LOUISIANA STATE UNIVERSITY & AGRICULTURAL & MECHANICAL COLLEGE','OHIO STATE UNIVERSITY','UNIVERSITY OF MIAMI',
'UNIVERSITY OF SOUTHERN CALIFORNIA','UNIVERSITY OF TEXAS - AUSTIN']

'''

# write a function to identify FBS Champion Schools

def champions(row):
    
    champs = ['UNIVERSITY OF ALABAMA','AUBURN UNIVERSITY','UNIVERSITY OF FLORIDA','FLORIDA STATE UNIVERSITY',
              'LOUISIANA STATE UNIVERSITY & AGRICULTURAL & MECHANICAL COLLEGE','OHIO STATE UNIVERSITY',
              'UNIVERSITY OF MIAMI','UNIVERSITY OF SOUTHERN CALIFORNIA','UNIVERSITY OF TEXAS - AUSTIN']
    
    if row['School'] in champs:
        y = 1
    else:
        y = 0
    return y

data_2015['fbs_champ'] = data_2015.apply(champions,axis=1)

# filter the 2015 dataframe for FBS champion schools

data_2015_champ = data_2015[(data_2015['fbs_champ'] == 1)]
print data_2015_champ

      OPE ID                             School State  Zip Code  \
6     100900                  AUBURN UNIVERSITY    AL     36849   
206   132800  UNIVERSITY OF SOUTHERN CALIFORNIA    CA     90089   
288   148900           FLORIDA STATE UNIVERSITY    FL     32306   
315   153500              UNIVERSITY OF FLORIDA    FL     32611   
316   153600                UNIVERSITY OF MIAMI    FL     33146   
1265  309000              OHIO STATE UNIVERSITY    OH     43210   
1640  365800       UNIVERSITY OF TEXAS - AUSTIN    TX     78712   

            School Type  dls_rec  dls_nlo     dls_dlo  dls_nd      dls_dd  \
6                Public     5157     5157  20947371.0    5196  11157280.0   
206   Private-Nonprofit     4954     4955  11871728.0    4955  11871728.0   
288              Public     7640     7640  16790741.0    7765  16680504.0   
315              Public     5903     5983  24283327.0    5983  13350000.0   
316   Private-Nonprofit     1941     1979   7751470.0    1979   4047601.0   
1

The above dataframe looks complete. Note that the absence of Alabama and Louisiana State from the dataframe is to be expected: both of these schools were filtered out in the initial 'zip code' filtering requested up front (i.e. no schools with zips ending in 3,5, or 7).

In [24]:
# Answer the question: the field of interest is 'dlgp_dlo' (i.e. Direct Loan Graduate Plus total loan amount in $)
# The question calls for the sum of this field for all FBS schools in 2015 - use the pandas 'sum' function

data_2015_champ['dlgp_dlo'].sum()

193095618.0

### QUESTION 5 ANSWER

The sum for the field of interest is $193,095,618